# Hitchhiker's generation with RNN using Keras

Inspired by http://karpathy.github.io/2015/05/21/rnn-effectiveness/

H2g2 plain text from http://www.clearwhitelight.org/hitch/hhgttg.txt

## Configs

In [77]:
seq_len = 50
batch_size = 50
iterations = 100

## Setup

In [127]:
import time
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import LSTM
import numpy as np
from sklearn.feature_extraction import DictVectorizer

In [61]:
f = open('./h2g2.txt')
text = f.read()

In [65]:
print(text[1000:2000])

tches. 

Many were increasingly of the opinion that they'd all made a big mistake in coming down from the trees in the first place. And some said that even the trees had been a bad move, and that no one should ever have left the oceans. 

And then, one Thursday, nearly two thousand years after one man had been nailed to a tree for saying how great it would be to be nice to people for a change, one girl sitting on her own in a small cafe in Rickmansworth suddenly realized what it was that had been going wrong all this time, and she finally knew how the world could be made a good and happy place. This time it was right, it would work, and no one would have to get nailed to anything. 

Sadly, however, before she could get to a phone to tell anyone- about it, a terribly stupid catastrophe occurred, and the idea was lost forever. 

This is not her story. 

But it is the story of that terrible stupid catastrophe and some of its consequences. 

It is also the story of a book, a book called Th

In [141]:
vec = DictVectorizer()

In [159]:
letters = list(text)
text_array = vec.fit_transform([{l:1} for l in letters])

In [160]:
text_array.shape

(272157, 83)

Looks about right, we have 272157 characters in the book and there are 83 unique characters. This one-hot encoding looks accurate.

## Helper functions

In [167]:
def next_batch(data, seq_len, batch_size):
    x_batch = []
    y_batch = []
    for _ in range(batch_size):
        i = np.random.randint(0, data.shape[0]-seq_len)
        x_i = data[i:(i+seq_len)].toarray()
        y_i = data[i+seq_len].toarray()
        x_batch.append(list(x_i))
        y_batch.append(y_i)
    return np.array(x_batch), np.array(y_batch)

In [171]:
# test out next_batch function
x_batch, y_batch = next_batch(text_array, 20, 5)
x_batch.shape

(5, 20, 83)

The function looks good. It generates a 5 records, where each record is 20 letters, where each letter is encoded as an array with length of 83.

## Define network

In [172]:
model = Sequential()
model.add(LSTM(32, input_shape=(seq_len, text_array.shape[1])))
model.add(Flatten())
model.add(Dense(text_array.shape[1], activation='softmax'))

ValueError: Input 0 is incompatible with layer flatten_1: expected ndim >= 3, found ndim=2

## Train

In [124]:
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

In [122]:
x_batch, y_batch = next_batch(text, 20, 5)
x_valid, y_valid = next_batch(text, 20, 5)

In [ ]:
t0 = time.time()
for _ in iterations:
    model.train_on_batch(x_batch, y_batch)
    model.test_on_batch(x_valid, y_valid)
print "training time = {}".format(time.time() - t0)